In [ ]:
# @title
# %% [markdown]
# # WanGP v5.41 - Complete Cloud Installation with Enhanced Debugging & Dual Share
#
# ## 🎬 ALL Features from v5.41 Including:
# - **Dual Public Access**: Gradio --share + ngrok for maximum reliability
# - **Full Debug Output**: Complete verbose logging of all operations
# - **Robust Directory Management**: Error-proof workspace handling
# - **All Models**: Wan, Hunyuan, LTX, VACE (1.3B & 14B), MoviiGen, etc.
# - **Queue System**: Stack multiple generation tasks
# - **Video Settings Management**: Save/load/reuse video settings (v5.3)
# - **Complete Error Handling**: Bulletproof against common issues

# %% [markdown]
# ## 1. Workspace Setup & Configuration

# %%
# ====================================
# 🔧 USER CONFIGURATION SECTION
# ====================================

# NGROK CONFIGURATION (Get token from: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = ""  # <- PASTE YOUR NGROK TOKEN HERE (Leave empty to use Gradio share only)

# WORKSPACE CONFIGURATION
WORKSPACE_NAME = "WanGP_Workspace"  # Main workspace directory
FORCE_CLEAN_INSTALL = False  # Set True to delete existing workspace
AUTO_FIX_CONFLICTS = True  # Automatically resolve directory conflicts

# ADVANCED SETTINGS
DEBUG_LEVEL = 2  # 0=minimal, 1=normal, 2=verbose (shows everything)
ENABLE_NGROK = True  # Use ngrok as backup/primary share method
NGROK_REGION = "us"  # us, eu, ap, au, sa, jp, in
MONITOR_RESOURCES = True  # Show real-time GPU/CPU/RAM usage
AUTO_RESTART_ON_FAIL = True  # Automatically retry if launch fails
ENABLE_UPSAMPLING = True  # Enable temporal/spatial upsampling features
DOWNLOAD_ALL_LORAS = True  # Download all essential loras

print("="*80)
print("🎮 WANGP v5.41 ROBUST INSTALLATION")
print("="*80)
print(f"Workspace: {WORKSPACE_NAME}")
print(f"Clean Install: {'YES - Will delete existing' if FORCE_CLEAN_INSTALL else 'NO - Will reuse/update'}")
print(f"Auto-fix: {'ENABLED' if AUTO_FIX_CONFLICTS else 'DISABLED'}")
print(f"Debug Level: {DEBUG_LEVEL} ({'Verbose' if DEBUG_LEVEL == 2 else 'Normal' if DEBUG_LEVEL == 1 else 'Minimal'})")
print(f"Ngrok: {'ENABLED' if ENABLE_NGROK and NGROK_AUTH_TOKEN else 'DISABLED'}")
print(f"Ngrok Token: {'✅ SET' if NGROK_AUTH_TOKEN else '❌ NOT SET (will use Gradio share)'}")
print("="*80)

## 2. Robust Directory Management System

In [ ]:
import os
import sys
import subprocess
import torch
import platform
import psutil
import json
import time
import socket
import shutil
from datetime import datetime
from pathlib import Path

def safe_remove_directory(path):
    """Safely remove directory with error handling"""
    if os.path.exists(path):
        try:
            if os.path.islink(path):
                os.unlink(path)
            else:
                shutil.rmtree(path)
            return True
        except Exception as e:
            print(f"⚠️ Could not remove {path}: {e}")
            return False
    return True

def ensure_clean_workspace():
    """Create a clean workspace directory"""
    current_dir = os.getcwd()
    workspace_path = os.path.join(current_dir, WORKSPACE_NAME)

    print("="*80)
    print("📁 WORKSPACE MANAGEMENT")
    print("="*80)
    print(f"Current directory: {current_dir}")
    print(f"Target workspace: {workspace_path}")

    # Handle existing workspace
    if os.path.exists(workspace_path):
        if FORCE_CLEAN_INSTALL:
            print(f"\n🧹 Force clean enabled - removing existing workspace...")
            if safe_remove_directory(workspace_path):
                print("✅ Existing workspace removed")
            else:
                print("❌ Could not remove workspace - will work around it")
        else:
            print(f"\n📁 Workspace exists - checking contents...")

            # Check if it contains WanGP
            wangp_path = os.path.join(workspace_path, "Wan2GP")
            if os.path.exists(wangp_path):
                print(f"✅ Found existing WanGP installation")

                # Check if it's a valid repo
                if os.path.exists(os.path.join(wangp_path, ".git")):
                    print("✅ Valid git repository found")
                    return workspace_path, wangp_path, "update"
                else:
                    print("⚠️ Directory exists but not a git repo")
                    if AUTO_FIX_CONFLICTS:
                        print("🔧 Auto-fix enabled - will clean and re-clone")
                        safe_remove_directory(wangp_path)
                        return workspace_path, wangp_path, "clone"
            else:
                print("📂 Empty workspace - will create WanGP inside")
                return workspace_path, os.path.join(workspace_path, "Wan2GP"), "clone"

    # Create new workspace
    print(f"\n📁 Creating new workspace: {WORKSPACE_NAME}")
    os.makedirs(workspace_path, exist_ok=True)

    return workspace_path, os.path.join(workspace_path, "Wan2GP"), "clone"

# Setup workspace
workspace_dir, repo_path, action = ensure_clean_workspace()

# Change to workspace
os.chdir(workspace_dir)
print(f"\n✅ Working in: {os.getcwd()}")
print(f"Action: {action}")
print("="*80)

## 3. System Diagnostics with Error Handling

In [ ]:
print("="*80)
print("🔍 COMPREHENSIVE SYSTEM DIAGNOSTICS")
print("="*80)

def safe_get_info(func, default="Unknown"):
    """Safely get system info with fallback"""
    try:
        return func()
    except:
        return default

# Basic system info
print(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"Platform: {safe_get_info(lambda: f'{platform.system()} {platform.release()}')}")
print(f"Python: {sys.version.split()[0]}")
print(f"Workspace: {workspace_dir}")

# Detect environment
env_indicators = {
    'Google Colab': lambda: 'google.colab' in str(get_ipython()),
    'Kaggle': lambda: 'KAGGLE_URL_BASE' in os.environ,
    'Lightning.ai': lambda: 'LIGHTNING_CLOUD_URL' in os.environ,
    'Vast.ai': lambda: os.path.exists('/opt/bin/nvidia-smi'),
    'Paperspace': lambda: 'PS_API_KEY' in os.environ
}

detected_env = "Unknown"
for env_name, check_func in env_indicators.items():
    if safe_get_info(check_func, False):
        detected_env = env_name
        break

print(f"Environment: {detected_env}")

# Resource info
print(f"\n[RESOURCES]")
print(f"CPU Cores: {safe_get_info(lambda: psutil.cpu_count(logical=False), 'Unknown')}")
mem = safe_get_info(lambda: psutil.virtual_memory(), None)
if mem:
    print(f"RAM: {mem.total/(1024**3):.1f} GB total, {mem.available/(1024**3):.1f} GB available")
else:
    print("RAM: Could not detect")

# GPU Detection with comprehensive error handling
print(f"\n[GPU DETECTION]")
gpu_available = False
gpu_info = "No GPU"
gpu_memory = 0
gpu_generation = 'none'

try:
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        print(f"✅ CUDA Available: True")
        print(f"GPU Count: {gpu_count}")
        print(f"CUDA Version: {torch.version.cuda}")

        # Get primary GPU info
        gpu_info = torch.cuda.get_device_name(0)
        props = torch.cuda.get_device_properties(0)
        gpu_memory = props.total_memory / (1024**3)

        print(f"Primary GPU: {gpu_info}")
        print(f"VRAM: {gpu_memory:.2f} GB")

        # Determine generation
        gpu_name_lower = gpu_info.lower()
        if any(x in gpu_name_lower for x in ['5090', '5080', '5070', '5060', '5050']):
            gpu_generation = 'rtx50xx'
            pytorch_version = "2.7.0"
            cuda_index = "cu128"
        elif any(x in gpu_name_lower for x in ['a100', 'a6000', 'a40', 'v100']):
            gpu_generation = 'datacenter'
            pytorch_version = "2.6.0"
            cuda_index = "cu124"
        else:
            gpu_generation = 'standard'
            pytorch_version = "2.6.0"
            cuda_index = "cu124"

        print(f"Generation: {gpu_generation}")
        gpu_available = True

    else:
        print("❌ CUDA Not Available")

except Exception as e:
    print(f"❌ GPU Detection Error: {e}")

if not gpu_available:
    print("\n🔧 GPU TROUBLESHOOTING:")
    print("1. Google Colab: Runtime > Change runtime type > GPU")
    print("2. Other platforms: Ensure GPU instance selected")
    print("3. Try: !nvidia-smi to check GPU status")

    # Still allow CPU execution for testing
    print("\n⚠️ Continuing with CPU mode (very slow)")
    gpu_info = "CPU Mode"
    gpu_memory = 0
    pytorch_version = "2.6.0"
    cuda_index = "cu124"

print("="*80)

## 4. Repository Management with Conflict Resolution

In [ ]:
print("="*80)
print("📦 REPOSITORY MANAGEMENT")
print("="*80)

def run_command_safe(cmd, shell=True, capture_output=True, timeout=300):
    """Run command with timeout and error handling"""
    try:
        result = subprocess.run(
            cmd,
            shell=shell,
            capture_output=capture_output,
            text=True,
            timeout=timeout
        )
        return result.returncode, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return -1, "", "Command timed out"
    except Exception as e:
        return -1, "", str(e)

def clone_or_update_repo():
    """Handle repository cloning/updating with conflict resolution"""
    repo_url = "https://github.com/remphanstar/WanBook.git"

    if action == "update":
        print(f"[1/3] Updating existing repository...")
        os.chdir(repo_path)

        # Check git status
        ret, stdout, stderr = run_command_safe("git status --porcelain")
        if ret == 0 and stdout.strip():
            print("⚠️ Local changes detected")
            if AUTO_FIX_CONFLICTS:
                print("🔧 Auto-fix: Stashing local changes...")
                run_command_safe("git stash")

        # Pull updates
        ret, stdout, stderr = run_command_safe("git pull")
        if ret == 0:
            print("✅ Repository updated")
        else:
            print(f"⚠️ Update failed: {stderr}")
            print("Will attempt fresh clone...")
            os.chdir(workspace_dir)
            safe_remove_directory(repo_path)
            action = "clone"

    if action == "clone":
        print(f"[1/3] Cloning repository from GitHub...")
        print(f"URL: {repo_url}")

        # Ensure target doesn't exist
        if os.path.exists(repo_path):
            print(f"⚠️ Target exists, removing: {repo_path}")
            safe_remove_directory(repo_path)

        # Clone with progress
        start_time = time.time()
        ret, stdout, stderr = run_command_safe(
            f"git clone {repo_url} {os.path.basename(repo_path)}",
            timeout=600  # 10 minutes max
        )

        elapsed = time.time() - start_time

        if ret == 0:
            print(f"✅ Repository cloned in {elapsed:.1f}s")
        else:
            print(f"❌ Clone failed: {stderr}")

            # Fallback: try wget
            print("🔄 Trying alternative download method...")
            zip_url = "https://github.com/remphanstar/WanBook/archive/refs/heads/main.zip"
            ret, stdout, stderr = run_command_safe(f"wget -O wangp.zip {zip_url}")

            if ret == 0:
                print("📦 Downloaded as ZIP, extracting...")
                ret, stdout, stderr = run_command_safe("unzip -q wangp.zip")
                if ret == 0:
                    # Rename extracted folder
                    extracted_name = "Wan2GP-main"
                    if os.path.exists(extracted_name):
                        os.rename(extracted_name, os.path.basename(repo_path))
                        print("✅ Repository extracted successfully")
                    else:
                        raise RuntimeError("Could not extract repository")
                else:
                    raise RuntimeError("Failed to extract ZIP")
            else:
                raise RuntimeError("All download methods failed")

# Execute repository setup
try:
    clone_or_update_repo()
except Exception as e:
    print(f"❌ Repository setup failed: {e}")
    print("\n🔧 Manual steps:")
    print("1. Check internet connection")
    print("2. Try: !git clone https://github.com/remphanstar/WanBook.git")
    print("3. Or download ZIP manually")
    raise

# Change to repo directory
print(f"\n[2/3] Entering repository directory...")
os.chdir(repo_path)
print(f"Working directory: {os.getcwd()}")

# Verify repository
print(f"\n[3/3] Verifying repository integrity...")
required_files = {
    'wgp.py': 'Main application',
    'requirements.txt': 'Dependencies',
    'CHANGELOG.md': 'Version history'
}

missing_files = []
for file, desc in required_files.items():
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024
        print(f"  ✅ {file:<20} ({size:>6.1f} KB)")
    else:
        print(f"  ❌ {file:<20} MISSING")
        missing_files.append(file)

if missing_files:
    print(f"\n❌ Repository incomplete! Missing: {', '.join(missing_files)}")
    if AUTO_FIX_CONFLICTS:
        print("🔧 Auto-fix: Attempting re-download...")
        os.chdir(workspace_dir)
        safe_remove_directory(repo_path)
        # Try again
        clone_or_update_repo()
        os.chdir(repo_path)
    else:
        raise FileNotFoundError(f"Missing critical files: {missing_files}")

print("\n✅ Repository ready")
print("="*80)